## RNN을 통해 성별 예측하기

In [17]:
import pandas as pd
import torch.nn as nn

In [18]:
df = pd.read_csv('../data/name_gender_filtered.csv')

#### 전처리

In [19]:
unique_chars = set() #중복되지 않는 문자들을 저장할 집합

for name in df['Name']:
    unique_chars.update(name)
unique_chars = sorted(list(unique_chars)) #문자들을 정렬
unique_chars = ''.join(unique_chars) #문자들을 하나의 문자열로 결합
print(unique_chars)

abcdefghijklmnopqrstuvwxyz


#### 원핫 인코딩

In [20]:
n_letters = len(unique_chars) #26개의 문자들을 가지고 있음

def nameToTensor(name):
    tensor = torch.zeros(len(name), n_letters) #크기: [4, 26]
    for char_idx, char in enumerate(name):
        #print(char_idx, char)∆
        letter_idx = unique_chars.find(char)
        #print(char, letter_idx)
        assert letter_idx != -1, f"char is {name}, {char}"
        tensor[char_idx][letter_idx] = 1
    return tensor

In [21]:
nameToTensor('john')

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])

### 모델 정의

In [24]:
#i
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.hidden_size = hidden_size
        # input to hidden size 정의(입력크기, 출력크기)
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        # input to output size 정의
        self.i2o = nn.Linear(hidden_size, output_size)
        

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        #활성화 함수로 tanh 사용(하이퍼볼릭 탄젠트)
        hidden = torch.tanh(self.i2h(combined))
        output = self.i2o(hidden)

        return output, hidden
        

    def get_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [25]:
n_hidden = 32
rnn_model = MyRNN(n_letters, n_hidden, 2)

### 학습

In [26]:
import torch 
from torch.optim import Adam, SGD

In [27]:
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(rnn_model.parameters(), lr=0.001)

In [29]:
rnn_model.train()

MyRNN(
  (i2h): Linear(in_features=58, out_features=32, bias=True)
  (i2o): Linear(in_features=32, out_features=2, bias=True)
)

In [58]:
gen2num = {'F':0, 'M':1}
num2gen = {0:'F', 1:'M'}

for epoch_idx in range(100):
    # CSV 데이터를 섞음 데이터의 순서를 무작위로 섞어서 학습 시 편향을 방지
    # frac=1은 전체 데이터의 100%를 샘플링
    # 섞인 후의 인덱스를 0부터 다시 순차적으로 설정
    suffled_df = df.sample(frac=1).reset_index(drop=True) 
    #print(suffled_df)

    total_loss = 0 #손실값을 저장할 변수
    correct_predictions = 0 #정답을 맞춘 횟수를 저장할 변수
    total_predictions = 0 #전체 예측 횟수를 저장할 변수

    for index, row in suffled_df.iterrows():
        input_tensor = nameToTensor(row['Name'])
        #print(f"Name: {row['Name']}, input_tensor: {input_tensor}")

        # 성별을 숫자로 변환 - 정답 레이블로 사용
        target_tensor = torch.tensor([gen2num[row['Gender']]], dtype=torch.long)
        #print(target_tensor)
        hidden = rnn_model.get_hidden()
        
        rnn_model.zero_grad() #기울기 초기화

        for char_idx in range(input_tensor.size()[0]):
            char_tensor = input_tensor[char_idx] #한 문자를 텐서로 변환
             #예측값과 은닉 상태를 반환 #None: 차원을 추가하여 2차원 텐서로 변환
            output, hidden = rnn_model(char_tensor[None,:], hidden)
            #print(output, hidden)

        loss = loss_fn(output, target_tensor) #손실값 계산
        loss.backward()
        optimizer.step()

        total_loss += loss.item() #손실값을 누적
        predicted_index = torch.argmax(output[0]).item() #모델의 출력에서 가장 큰 값의 인덱스를 가져오는 코드
        #print(target_tensor)
        correct_predictions += (predicted_index == target_tensor).sum().item()  
        total_predictions += 1
    
    average_loss = total_loss / total_predictions #평균 손실값 계산
    accuracy = 100 * correct_predictions / total_predictions #정확도 = 100 * 정답을 맞춘 횟수 / 전체 예측 횟수
    print(f'Epoch: {epoch_idx}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')










Epoch: 0, Loss: 0.3897, Accuracy: 82.72%
Epoch: 1, Loss: 0.3802, Accuracy: 83.19%


### 테스트

In [76]:
test_name = 'elsa'
test_name_tensor = nameToTensor(test_name)

rnn_model.eval() #모델을 평가 모드로 전환
hidden = rnn_model.get_hidden()

for char_idx in range(test_name_tensor.size()[0]):
    char_tensor = test_name_tensor[char_idx]
    output, hidden = rnn_model(char_tensor[None,:], hidden)
predicted_index = torch.argmax(output, 1).item()
print(num2gen[predicted_index])

F
